<a href="https://colab.research.google.com/github/bogdanalexandrutreica/llama_index/blob/main/data_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3
!pip install pydantic==1.10.9
!pip installl lama_index
!pip install llama-index-embeddings-bedrock==0.1.4
!pip install llama-index-llms-bedrock==0.1.5
!pip install llama-index-vector-stores-postgres==0.1.5
!pip install langchain==0.0.333
!pip install PyMuPDF

In [ ]:
!pip install llama-index-readers-file

In [ ]:
import os

os.mkdir("./data")

In [ ]:
import fitz

doc = fitz.open("./data/test.pdf")
full_pdf_content = ""
for page in range(doc.page_count):
  pg = doc.load_page(page)
  text = pg.get_text()
  full_pdf_content += text

print(full_pdf_content)

In [ ]:
from llama_index.core.node_parser.text.sentence import SentenceSplitter
from llama_index.core.schema import Document
import pprint as pp

chunk_size = 1024
chunk_overlap = 100

splitter = SentenceSplitter(
        chunk_size=chunk_size,
        # chunk_size=512,
        chunk_overlap=chunk_overlap,
    )

docs = splitter.get_nodes_from_documents(
        [
            Document(
                #text=docs
                text=full_pdf_content,
                metadata={
                    "page": 0,
                    #"file_name": f"{metadata_file_name}",
                    "source": "text",
                    "document_id": f"some_id",
                },
                excluded_embed_metadata_keys=["page", "source", "document_id"],
                excluded_llm_metadata_keys=["page", "source", "document_id"],
            )
        ]
    )
pp.pprint(docs[0])

In [ ]:
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.llms.bedrock import Bedrock
from llama_index.core.settings import Settings
import boto3
import asyncio
import os


os.environ["AWS_REGION"] = "us-west-2"
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_SESSION_TOKEN"] = ""

embed_model_id = "amazon.titan-embed-text-v1"
llm_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

BEDROCK_CLIENT = boto3.client("bedrock-runtime", "us-west-2")

llm = Bedrock(
    model=llm_model_id, client=BEDROCK_CLIENT, aws_region_name="us-west-2")
embed_model = BedrockEmbedding(model=embed_model_id, client=BEDROCK_CLIENT)

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
from llama_index.core.ingestion.pipeline import IngestionPipeline
from llama_index.core.extractors import QuestionsAnsweredExtractor
import asyncio

pipeline = IngestionPipeline(
          transformations=[
              QuestionsAnsweredExtractor(questions=2)
           ]
      )
nodes = await pipeline.arun(nodes = docs)
pp.pprint(nodes[0])

In [ ]:
from llama_index.core import VectorStoreIndex

index =  VectorStoreIndex(nodes)

print(index)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query("What is this story about?")

print(response)

In [ ]:
for n in response.source_nodes:
  print(n)